# Artifact Table 1-B for Phase Moducation Side Channel Analysis and JTAG Attacks

This is part of a series of artifacts. Note these notebooks are designed to run on Google Colab so have been simplified from those available in the [full repository](https://github.com/colinoflynn/phase-modulation-sca).

To run these notebooks, simply run from top to bottom.

**WARNING**: Some steps, such as the extraction, can be very slow. Try not to forget you started the process as CoLab will kill unattended notebooks eventually, and you'll have to start again.

In [ ]:
!sudo apt-get install zstd

In [ ]:
!wget https://zenodo.org/records/13359322/files/baseline_traces_ttest.tar.zst?download=1

In [ ]:
!tar -xf baseline_traces_ttest.tar.zst?download=1

In [ ]:
!rm -rf baseline_traces_ttest.tar.zst?download=1

## T-Test Results

The following recreats the T-Test results for Table 1.

In [ ]:
import os
import numpy as np
def plot_t(t_val, N, titledata=""):
    import matplotlib.pylab as plt
    ldata = len(t_val)
    plt.plot(t_val)
    plt.plot([0, ldata], [4.5, 4.5], 'k--')
    plt.plot([0, ldata], [-4.5, -4.5], 'k--')
    plt.title("TVLA Results, N=%d, %s"%(N, titledata))
    plt.xlabel("Sample Number")
    plt.ylabel("T-Test Value")

#Set root data dir, not "mixers"
data_dir = ""

def npload(filename):
    return np.load(os.path.join(data_dir, filename), allow_pickle=True)

def nptload(prefix):
    group1 = npload(prefix + "_tvla_group1.npy")
    group2 = npload(prefix + "_tvla_group2.npy")
    N = round((len(group1) + len(group2)) / 2)
    return group1, group2, N

def npsave(filename, array):
    np.save(os.path.join(data_dir, filename), array)

def nptsave(prefix, group1, group2):
    npsave(prefix + "_tvla_group1.npy", group1)
    npsave(prefix + "_tvla_group2.npy", group2)


In [ ]:
!ls -lh baseline_traces_ttest/

In [ ]:
files = [
         [r"baseline_traces_ttest/baseline120mhz_sync_120msps_ttest_10k", 100, 10000, 0.01, 0.1], #None, None],
         [r"baseline_traces_ttest/baseline120mhz_sync_60msps_ttest_10k", 200, 5000, 0.01, 0.1], # None, None],
         [r"baseline_traces_ttest/baseline120mhz_async_120msps_ttest_10k", 200, 10000, 0.01, 0.1], # None],
         [r"baseline_traces_ttest/baseline120mhz_async_60msps_ttest_10k", 100, 5000, 0.01, 0.1], # None, None],
         [r"baseline_traces_ttest/baseline15mhz_sync_120msps_ttest_10k", 1000, -1, 0.01, 0.1], # None, None],
         [r"baseline_traces_ttest/baseline15mhz_sync_60msps_ttest_10k", 500, -1, 0.01, 0.1], # None, None],
         [r"baseline_traces_ttest/baseline15mhz_async_120msps_ttest_10k", 1000, -1, 0.01, 0.1], # None, None],
         [r"baseline_traces_ttest/baseline15mhz_async_60msps_ttest_10k", 500, -1, 0.01, 0.1], # None, None],
        ]

In [ ]:
from scipy.stats import ttest_ind
import matplotlib.pylab as plt
from scipy import signal

def find_max_t(file, start=0, end=-1, hpf=None, lpf=None, resync_jtag=False):
    group1, group2, N = nptload(file)

    if resync_jtag:
        first = []
        for g in group1:
            first.append(g[0])
        avg = np.mean(first)
        group1resync = []
        for g in group1:
            if g[0] > avg:
                group1resync.append(g[:-1])
            else:
                group1resync.append(g[1:])

        group2resync = []
        for g in group2:
            if g[0] > avg:
                group2resync.append(g[:-1])
            else:
                group2resync.append(g[1:])

        group1 = group1resync
        group2 = group2resync

    if hpf:
        sos = signal.butter(5, hpf, 'highpass', output='sos')
        group1 = signal.sosfilt(sos, group1)
        group2 = signal.sosfilt(sos, group2)

    if lpf:
        sos = signal.butter(5, lpf, 'lowpass', output='sos')
        group1 = signal.sosfilt(sos, group1)
        group2 = signal.sosfilt(sos, group2)

    t_val = ttest_ind(group1[start:end], group2[start:end], axis=0, equal_var=False)[0]
    max_t = max(abs(t_val))
    return max_t, t_val
for f in files:
    max_t, t_val = find_max_t(f[0], start=f[1], end=f[2], hpf=f[3], lpf=f[4])
    print("{:s}: {:f}".format(f[0], max_t))

**WARNING** - the above seems to crash on colab due to memory sometimes!

These baseline values can be found in **Table 1** in the paper, reproduced here in Markdown (not exactly perfect):

| CPU Freq | Asynchronous Sampling |       |         |       | Synchronous Sampling |      |         |      |
|----------|-----------------------|-------|---------|-------|----------------------|------|---------|------|
|          | 120 MS/s              |       | 60 MS/s |       | 120 MS/S             |      | 60 MS/s |      |
|          | |TVLA|                | CPA   | |TVLA|  | CPA   | |TVLA|               | CPA  | |TVLA|  | CPA  |
| 120 MHz  | 31.3                  | 28375 | 4.2     | 66750 | 43.9                 | 750  | 28.8    | 1350 |
| 15 MHz   | 5.9                   | 19900 | 5.0     | 22800 | 24.0                 | 3250 | 25.8    | 3600 |

The output of the above should match the TVLA values as described by the filename.

